In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [203]:
dataset = pd.read_csv('breast/breast.csv')
dataset

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [204]:
# dataset.plot(x='radius_mean', y='smoothness_mean', style='o')
# dataset.plot(x='smoothness_mean', y='compactness_mean', style='o')
# dataset.plot(x='smoothness_mean', y='concavity_mean', style='o')


In [205]:
processed_data = dataset.drop(columns=['id', 'diagnosis', 'Unnamed: 32'])
#processed_data

In [206]:
trainset = processed_data.iloc[:455, :]
testset = processed_data.iloc[455:, :]

trainset

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,11.87,21.54,76.83,432.0,0.06613,0.10640,0.08777,0.02386,0.1349,0.06612,...,12.79,28.18,83.51,507.2,0.09457,0.3399,0.3218,0.08750,0.2305,0.09952
451,19.59,25.00,127.70,1191.0,0.10320,0.09871,0.16550,0.09063,0.1663,0.05391,...,21.44,30.96,139.80,1421.0,0.15280,0.1845,0.3977,0.14660,0.2293,0.06091
452,12.00,28.23,76.77,442.5,0.08437,0.06450,0.04055,0.01945,0.1615,0.06104,...,13.09,37.88,85.07,523.7,0.12080,0.1856,0.1811,0.07116,0.2447,0.08194
453,14.53,13.98,93.86,644.2,0.10990,0.09242,0.06895,0.06495,0.1650,0.06121,...,15.80,16.93,103.10,749.9,0.13470,0.1478,0.1373,0.10690,0.2606,0.07810


In [207]:
REPLACE_COUNT = trainset.size*0.1               # Set the number of missing values
NAN = 0

missing_data_train = trainset.to_numpy()
full_data_train = missing_data_train.copy()     # Create a copy of the same trainset

missing_data_train.flat[np.random.choice(missing_data_train.size, int(REPLACE_COUNT), replace=False)] = NAN

missing_data_test = testset.to_numpy()
full_data_test = missing_data_test.copy()

missing_data_test.flat[np.random.choice(missing_data_test.size, int(REPLACE_COUNT*0.2), replace=False)] = NAN

np.set_printoptions(precision=2, suppress=True)

print(missing_data_train)

[[ 17.99  10.38 122.8  ...   0.27   0.46   0.12]
 [  0.    17.77 132.9  ...   0.19   0.28   0.09]
 [ 19.69  21.25 130.   ...   0.24   0.36   0.09]
 ...
 [ 12.    28.23  76.77 ...   0.     0.24   0.08]
 [ 14.53  13.98  93.86 ...   0.11   0.26   0.08]
 [ 12.62   0.    80.62 ...   0.1    0.33   0.07]]


In [255]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.enc1 = nn.Linear(in_features=30, out_features=16)
        self.enc2 = nn.Linear(in_features=16, out_features=8)
        self.enc3 = nn.Linear(in_features=8, out_features=4)

        self.dec1 = nn.Linear(in_features=4, out_features=8)
        self.dec2 = nn.Linear(in_features=8, out_features=16)
        self.dec3 = nn.Linear(in_features=16, out_features=30)

    def forward(self, x):
        x = F.leaky_relu(self.enc1(x))
        x = F.leaky_relu(self.enc2(x))
        x = F.leaky_relu(self.enc3(x))
        x = F.leaky_relu(self.dec1(x))
        x = F.leaky_relu(self.dec2(x))
        x = self.dec3(x)
        return x

In [256]:
net = Autoencoder().double()
print(net)

Autoencoder(
  (enc1): Linear(in_features=30, out_features=16, bias=True)
  (enc2): Linear(in_features=16, out_features=8, bias=True)
  (enc3): Linear(in_features=8, out_features=4, bias=True)
  (dec1): Linear(in_features=4, out_features=8, bias=True)
  (dec2): Linear(in_features=8, out_features=16, bias=True)
  (dec3): Linear(in_features=16, out_features=30, bias=True)
)


In [257]:
NUM_EPOCHS = 6000
LEARNING_RATE = 0.001
BATCH_SIZE = 455
NUM_FEATURES = 30
BATCH_SIZE_TEST = 114

In [258]:
x_train = torch.from_numpy(missing_data_train)
x_train = x_train.view(1, BATCH_SIZE, NUM_FEATURES)

y_train = torch.from_numpy(full_data_train)
y_train = y_train.view(1, BATCH_SIZE, NUM_FEATURES)

x_test = torch.from_numpy(missing_data_test)
x_test = x_test.view(1, BATCH_SIZE_TEST, NUM_FEATURES)

y_test = torch.from_numpy(full_data_test)
y_test = y_test.view(1, BATCH_SIZE_TEST, NUM_FEATURES)

x_train.shape

torch.Size([1, 455, 30])

In [259]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [260]:
def train(net):
    train_loss = []
    torch.set_printoptions(precision=3, sci_mode=False)
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        for missing_data, full_data in zip(x_train, y_train):
            optimizer.zero_grad()
            outputs = net(missing_data.double())
            loss = criterion(outputs, full_data)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if epoch == NUM_EPOCHS-1:
                for i in range(len(missing_data)):
                    print("Input: ", missing_data[i])
                    print("Target: ", full_data[i])
                    print("Outputs: ", outputs[i])

        loss = running_loss / len(x_train)
        train_loss.append(loss)

        print('Epoch {} of {}, Train Loss: {:.3f}'
            .format(epoch+1, NUM_EPOCHS, loss))

    return train_loss


def test(net):

    net.eval()

    with torch.no_grad():
        test_loss = []
        running_loss = 0.0

        for missing_data, full_data in zip(x_test, y_test):
            outputs = net(missing_data.double())
            loss = criterion(outputs, full_data)
            running_loss += loss.item()
            for i in range(len(missing_data)):
                    print("Input: ", missing_data[i])
                    print("Target: ", full_data[i])
                    print("Outputs: ", outputs[i])

        loss = running_loss / len(x_test)
        test_loss.append(loss)
        print('Test Loss: {:.3f}'.format(loss))

        return test_loss

In [261]:
results = train(net)

7,     0.159,     0.056,     0.214,     0.655,
            1.606,    19.250,     0.005,     0.009,     0.009,     0.011,
            0.012,     0.002,    14.910,    19.310,    96.530,   688.900,
            0.103,     0.102,     0.063,     0.082,     0.214,     0.067],
       dtype=torch.float64)
Outputs:  tensor([    13.329,     17.472,     83.823,    604.514,      0.334,      0.119,
            -0.169,     -0.058,      0.279,      0.186,      0.615,      1.094,
             2.478,     31.821,      0.327,      0.187,      0.153,      0.123,
            -0.228,     -0.102,     13.797,     21.625,     92.385,    689.737,
             0.139,      0.292,      0.350,      0.088,      0.114,     -0.165],
       dtype=torch.float64, grad_fn=<SelectBackward>)
Input:tensor([   10.970,    17.200,    71.730,   371.500,     0.089,     0.111,
            0.095,     0.036,     0.149,     0.066,     0.257,     1.376,
            2.806,     0.000,     0.000,     0.046,     0.064,     0.018,
         

In [262]:
test_result = test(net)

           0.040,     0.013,     0.169,     0.066,     0.301,     1.879,
            0.000,    17.860,     0.011,     0.018,     0.040,     0.013,
            0.038,     0.005,     9.845,    25.050,    62.860,   295.800,
            0.110,     0.083,     0.080,     0.026,     0.243,     0.074],
       dtype=torch.float64)
Target:  tensor([    9.333,    21.940,    59.010,   264.000,     0.092,     0.056,
            0.040,     0.013,     0.169,     0.066,     0.301,     1.879,
            2.121,    17.860,     0.011,     0.018,     0.040,     0.013,
            0.038,     0.005,     9.845,    25.050,    62.860,   295.800,
            0.110,     0.083,     0.080,     0.026,     0.243,     0.074],
       dtype=torch.float64)
Outputs:  tensor([     7.902,     10.691,     40.596,    267.168,      0.058,      0.023,
             0.023,     -0.001,      0.114,      0.075,      0.212,      1.293,
             1.264,      9.364,      0.065,      0.110,     -0.002,      0.035,
            -0.006

In [263]:
torch.save(net, './breast_autoencoder2')

In [281]:
dataset = pd.read_csv('breast/breast.csv')
processed_data = dataset.drop(columns=['id', 'diagnosis', 'Unnamed: 32'])
processed_data.shape

(569, 30)

In [282]:
REPLACE_COUNT = processed_data.size*0.1               # Set the number of missing values
NAN = 0

input_set = processed_data.to_numpy()

input_set.flat[np.random.choice(input_set.size, int(REPLACE_COUNT), replace=False)] = NAN

np.set_printoptions(precision=2, suppress=True)

input_tensor = torch.from_numpy(input_set)
input_tensor = input_tensor.view(1, input_tensor.shape[0], NUM_FEATURES)

In [283]:
my_model = torch.load('breast_autoencoder2')
my_model

Autoencoder(
  (enc1): Linear(in_features=30, out_features=16, bias=True)
  (enc2): Linear(in_features=16, out_features=8, bias=True)
  (enc3): Linear(in_features=8, out_features=4, bias=True)
  (dec1): Linear(in_features=4, out_features=8, bias=True)
  (dec2): Linear(in_features=8, out_features=16, bias=True)
  (dec3): Linear(in_features=16, out_features=30, bias=True)
)

In [284]:
new_dataset = my_model(input_tensor.double())
torch.set_printoptions(precision=4, sci_mode=False)
new_dataset

tensor([[[    15.7127,     18.1389,     97.9401,  ...,      0.3665,
               0.0464,     -0.1968],
         [    22.7947,     27.2437,    158.2037,  ...,     -0.1132,
               0.5668,      0.3044],
         [    21.4896,     26.8871,    150.2514,  ...,      0.0352,
               0.3066,      0.0840],
         ...,
         [    12.9184,     10.3122,     74.4957,  ...,      0.0208,
               0.8839,      0.7814],
         [    22.7426,     23.0631,    151.9870,  ...,     -0.3621,
               1.6734,      1.3544],
         [     6.2583,      8.6464,     27.4653,  ...,      0.1873,
               0.2324,      0.0704]]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [285]:
new_dataset = new_dataset.view(new_dataset.shape[1], new_dataset.shape[2])
new_dataset

tensor([[    15.7127,     18.1389,     97.9401,  ...,      0.3665,
              0.0464,     -0.1968],
        [    22.7947,     27.2437,    158.2037,  ...,     -0.1132,
              0.5668,      0.3044],
        [    21.4896,     26.8871,    150.2514,  ...,      0.0352,
              0.3066,      0.0840],
        ...,
        [    12.9184,     10.3122,     74.4957,  ...,      0.0208,
              0.8839,      0.7814],
        [    22.7426,     23.0631,    151.9870,  ...,     -0.3621,
              1.6734,      1.3544],
        [     6.2583,      8.6464,     27.4653,  ...,      0.1873,
              0.2324,      0.0704]], dtype=torch.float64,
       grad_fn=<ViewBackward>)

In [286]:
np.savetxt('new_breast2.csv', new_dataset.detach().numpy())